In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [7]:
train_data_gen = ImageDataGenerator(rescale = 1./255)
validation_data_gen = ImageDataGenerator(rescale = 1./255)

In [8]:
train_generator = train_data_gen.flow_from_directory('/kaggle/input/emotion-detection-fer/train', target_size=(48, 48), batch_size=64, color_mode='grayscale', class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [10]:
validation_generator = validation_data_gen.flow_from_directory('/kaggle/input/emotion-detection-fer/test',target_size=(48,48),batch_size=64, color_mode='grayscale',class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [12]:
emotion_model = Sequential()

In [15]:
emotion_model.add(Conv2D(32, kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
emotion_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation='relu'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [17]:
emotion_model_info=emotion_model.fit_generator(train_generator, steps_per_epoch=28709//64, epochs=50, validation_data=validation_generator, validation_steps=7178//64)

/tmp/ipykernel_28/1514010088.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info=emotion_model.fit_generator(train_generator, steps_per_epoch=28709//64, epochs=50, validation_data=validation_generator, validation_steps=7178//64)


Epoch 1/50


2023-09-10 06:37:22.083411: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


448/448 [==============================] - 156s 328ms/step - loss: 1.9984 - accuracy: 0.2399 - val_loss: 1.8184 - val_accuracy: 0.2472
Epoch 2/50
448/448 [==============================] - 33s 74ms/step - loss: 1.8529 - accuracy: 0.2510 - val_loss: 1.8246 - val_accuracy: 0.2473
Epoch 3/50
448/448 [==============================] - 34s 75ms/step - loss: 1.8447 - accuracy: 0.2508 - val_loss: 1.8251 - val_accuracy: 0.2472
Epoch 4/50
448/448 [==============================] - 38s 85ms/step - loss: 1.8275 - accuracy: 0.2515 - val_loss: 1.8017 - val_accuracy: 0.2515
Epoch 5/50
448/448 [==============================] - 33s 73ms/step - loss: 1.8259 - accuracy: 0.2574 - val_loss: 1.8015 - val_accuracy: 0.2586
Epoch 6/50
448/448 [==============================] - 34s 76ms/step - loss: 1.7978 - accuracy: 0.2705 - val_loss: 1.7825 - val_accuracy: 0.3055
Epoch 7/50
448/448 [==============================] - 37s 83ms/step - loss: 1.7861 - accuracy: 0.2908 - val_loss: 1.7437 - val_accuracy: 0.3066
E

In [1]:
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

emotion_model.save_weights("emotion_model.h5")

NameError: name 'emotion_model' is not defined